In [1]:

from google.colab import drive
drive.mount('/content/drive')
import numpy as np 
import pandas as pd 
import tensorflow_hub as hub
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import Model, layers, Input
from tensorflow.keras.layers import *

from sklearn.metrics import confusion_matrix,f1_score,classification_report
from sklearn.preprocessing import LabelEncoder


use  = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") ## universal sentence encoder model

# from model import model
import pickle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
print('tf version:',tf.__version__)
print('tf_hub version:',hub.__version__)

tf version: 2.8.2
tf_hub version: 0.12.0


In [3]:
train_path = 'drive/MyDrive/Colab Notebooks/datasets/snli/snli_1.0_train.csv'
test_path = 'drive/MyDrive/Colab Notebooks/datasets/snli/snli_1.0_test.csv'
validation_path = 'drive/MyDrive/Colab Notebooks/datasets/snli/snli_1.0_dev.csv'

selected_columns = ['sentence1','sentence2','label1']

train_df = pd.read_csv(train_path,usecols = selected_columns)
train_df = train_df.rename(columns={"sentence1": "premise", "sentence2": "hypothesis", "label1": "label"})
test_df =  pd.read_csv(test_path,usecols = selected_columns)
test_df = test_df.rename(columns={"sentence1": "premise", "sentence2": "hypothesis", "label1": "label"})
validation_df =  pd.read_csv(validation_path,usecols = selected_columns)
validation_df = validation_df.rename(columns={"sentence1": "premise", "sentence2": "hypothesis", "label1": "label"})


In [4]:
LE = LabelEncoder()
LE.fit(train_df['label'])
train_df['label'] = enc_label = LE.transform(train_df['label'])
test_df['label'] = enc_label = LE.transform(test_df['label'])
validation_df['label'] = enc_label = LE.transform(validation_df['label'])


output = open('LE.pkl', 'wb')
pickle.dump(LE, output)
output.close()

pkl_file = open('LE.pkl', 'rb')
LE = pickle.load(pkl_file) 
pkl_file.close()

In [5]:
# conv_size = 10
conv_size = 40
drop_rate = 0.2
pool_size = 6

In [6]:
print(len(train_df))
train_df = train_df.dropna().reset_index(drop = True)
len(train_df)

550152


550146

In [7]:
def model():
    left_input = Input(shape = (), name = 'left_input', dtype = tf.string)
    right_input = Input(shape = (), name = 'right_input', dtype = tf.string)

    encoder_layer = hub.KerasLayer(use, trainable = True)

    left_encoder = encoder_layer(left_input)
    right_encoder = encoder_layer(right_input)
    subtract_layer = tf.keras.layers.Subtract()([left_encoder, right_encoder])
    subtract_layer = tf.math.abs(subtract_layer)

    left_encoder_wide = tf.expand_dims(left_encoder,axis=-1)
    right_encoder_wide = tf.expand_dims(right_encoder,axis=-1)
    subtract_wide = tf.expand_dims(subtract_layer,axis=-1)

    left_convo  = Conv1D(512, conv_size)(left_encoder_wide)
    left_pool = MaxPool1D(pool_size)(left_convo)

    right_convo  = Conv1D(512, conv_size)(right_encoder_wide)
    right_pool = MaxPool1D(pool_size)(right_convo)

    sub_convo = Conv1D(512, conv_size)(subtract_wide)
    sub_pool = MaxPool1D(pool_size)(sub_convo)

    left_pool = Flatten()(left_pool)
    right_pool = Flatten()(right_pool)
    sub_pool = Flatten()(sub_pool)



    concat_layer = tf.keras.layers.Concatenate(axis=-1)([left_pool, right_pool,sub_pool])


    d2_layer = Dense(512, activation = 'relu')(concat_layer)
    d2_layer = Dropout(drop_rate)(d2_layer)
    d3_layer = Dense(256, activation = 'relu')(d2_layer)
    d3_layer = Dropout(drop_rate)(d3_layer)
    d3_layer = Dense(64, activation = 'relu')(d2_layer)
    d3_layer = Dropout(drop_rate)(d3_layer)
    d4_layer = Dense(3, activation = 'softmax')(d3_layer)

    return Model(inputs = [left_input, right_input], outputs = d4_layer)

In [8]:
use_model = model()
use_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(
                                        # learning_rate = 0.006
    ),
    metrics= ['accuracy'])

history=use_model.fit(
    [train_df['premise'], train_df['hypothesis']],
    train_df['label'],
    epochs = 2,
    batch_size = 20,
    validation_split = 0.2
                    )


use_model.save_weights('drive/MyDrive/Colab Notebooks/snli/USE_snli_weights.h5')

In [9]:
use_model.load_weights('drive/MyDrive/Colab Notebooks/snli/USE_snli_weights.h5')

In [10]:
predictions = tf.math.top_k(use_model.predict([test_df['premise'], test_df['hypothesis']]),k=1)[1]

In [11]:
confusion_matrix(test_df['label'], predictions)

array([[2537,  339,  457],
       [  82, 2885,  366],
       [ 231,  618, 2485]])

In [12]:
print('F1 SCORE :',f1_score(test_df['label'], predictions, average='macro'))

F1 SCORE : 0.7910290031945397


In [14]:

print(classification_report(test_df['label'], predictions, target_names=LE.inverse_transform([0,1,2])))

               precision    recall  f1-score   support

contradiction       0.89      0.76      0.82      3333
   entailment       0.75      0.87      0.80      3333
      neutral       0.75      0.75      0.75      3334

     accuracy                           0.79     10000
    macro avg       0.80      0.79      0.79     10000
 weighted avg       0.80      0.79      0.79     10000

